## set up

1. create a workspace
2. create a warehouse

## dynamic data masking rules to columns 

==> FirstName varchar(50) MASKED WITH (FUNCTION = 'partial(1,"XXXXXXX",0)') NULL,

all queries are affected by the masking. 

Users who do not have explicit permissions to view confidential data see masked values in query results while users with explicit permission to view the data see it unobscured. 

There are four types of masks: 
- default, 
- email, 
- random 
- custom string

        CREATE TABLE dbo.Customers
        (   
            CustomerID INT NOT NULL,   
            FirstName varchar(50) MASKED WITH (FUNCTION = 'partial(1,"XXXXXXX",0)') NULL,     
            LastName varchar(50) NOT NULL,     
            Phone varchar(20) MASKED WITH (FUNCTION = 'default()') NULL,     
            Email varchar(50) MASKED WITH (FUNCTION = 'email()') NULL   
        );
        
        INSERT dbo.Customers (CustomerID, FirstName, LastName, Phone, Email) VALUES
        (29485,'Catherine','Abel','555-555-5555','catherine0@adventure-works.com'),
        (29486,'Kim','Abercrombie','444-444-4444','kim2@adventure-works.com'),
        (29489,'Frances','Adams','333-333-3333','frances0@adventure-works.com');
        
        SELECT * FROM dbo.Customers;


## Apply row-level security

Row-level security (RLS) can be used to limit access to rows based on the identity, 

or role of the user executing a query. 

1. restrict access to rows by creating a security policy and a security predicate defined as an inline table-valued function.

replace <username1>@<your_domain>.com with either a fictitious user name or a real one from your environment (Viewer role), and replace <username2>@<your_domain>.com with your user name (Admin role).

            CREATE TABLE dbo.Sales  
            (  
                OrderID INT,  
                SalesRep VARCHAR(60),  
                Product VARCHAR(10),  
                Quantity INT  
            );
                
            --Populate the table with 6 rows of data, showing 3 orders for each test user. 
            INSERT dbo.Sales (OrderID, SalesRep, Product, Quantity) VALUES
            (1, '<username1>@<your_domain>.com', 'Valve', 5),   
            (2, '<username1>@<your_domain>.com', 'Wheel', 2),   
            (3, '<username1>@<your_domain>.com', 'Valve', 4),  
            (4, '<username2>@<your_domain>.com', 'Bracket', 2),   
            (5, '<username2>@<your_domain>.com', 'Wheel', 5),   
            (6, '<username2>@<your_domain>.com', 'Seat', 5);  
                
            SELECT * FROM dbo.Sales;  

2. Create a new schema, a security predicate defined as a function, and a security policy.

            --Create a separate schema to hold the row-level security objects (the predicate function and the security policy)
            CREATE SCHEMA rls;
            GO
            
            /*Create the security predicate defined as an inline table-valued function.
            A predicate evaluates to true (1) or false (0). This security predicate returns 1,
            meaning a row is accessible, when a row in the SalesRep column is the same as the user
            executing the query.*/   
            --Create a function to evaluate who is querying the table
            CREATE FUNCTION rls.fn_securitypredicate(@SalesRep AS VARCHAR(60)) 
                RETURNS TABLE  
            WITH SCHEMABINDING  
            AS  
                RETURN SELECT 1 AS fn_securitypredicate_result   
            WHERE @SalesRep = USER_NAME();
            GO   
            /*Create a security policy to invoke and enforce the function each time a query is run on the Sales table.
            The security policy has a filter predicate that silently filters the rows available to 
            read operations (SELECT, UPDATE, and DELETE). */
            CREATE SECURITY POLICY SalesFilter  
            ADD FILTER PREDICATE rls.fn_securitypredicate(SalesRep)   
            ON dbo.Sales  
            WITH (STATE = ON);
            GO


## Implement column-level security

Column-level security allows you to designate which users can access specific columns in a table. 

It’s implemented by issuing a GRANT or DENY statement on a table specifying a list of columns and the user or role that can or cannot read them

1. create table

        CREATE TABLE dbo.Orders
        (   
            OrderID INT,   
            CustomerID INT,  
            CreditCard VARCHAR(20)      
        );   
        INSERT dbo.Orders (OrderID, CustomerID, CreditCard) VALUES
        (1234, 5678, '111111111111111'),
        (2341, 6785, '222222222222222'),
        (3412, 7856, '333333333333333');   
        SELECT * FROM dbo.Orders;


2. DENY permissions

        DENY SELECT ON dbo.Orders (CreditCard) TO [<username1>@<your_domain>.com];



## Configure SQL granular permissions using T-SQL
Fabric has a permissions model that allows you to control access to data at the workspace level, and at the item level. 

When you need more granular control of what users can do with securables in a Fabric warehouse, you can use the standard SQL data control language (DCL) commands GRANT,DENY and, REVOKE.

1. Create a stored procedure and a table. Then execute the procedure and query the table.

        CREATE PROCEDURE dbo.sp_PrintMessage
        AS
        PRINT 'Hello World.';
        GO   
        CREATE TABLE dbo.Parts
        (
            PartID INT,
            PartName VARCHAR(25)
        );
        
        INSERT dbo.Parts (PartID, PartName) VALUES
        (1234, 'Wheel'),
        (5678, 'Seat');
        GO
        
        /*Execute the stored procedure and select from the table and note the results you get
        as a member of the Workspace Admin role. Look for output from the stored procedure on 
        the 'Messages' tab.*/
        EXEC dbo.sp_PrintMessage;
        GO   
        SELECT * FROM dbo.Parts

2. DENY SELECT permissions on the table to a user who is a member of the Workspace Viewer role and GRANT EXECUTE on the procedure to the same user.

        DENY SELECT on dbo.Parts to [<username1>@<your_domain>.com];

        GRANT EXECUTE on dbo.sp_PrintMessage to [<username1>@<your_domain>.com];

* ==> user will see grant message procedure output but not the select output